## Data Ingestion -> Data Transformation (dividing into chunks) -> Embedding (text to vectors) -> VectorStoreDB (FAISS, CMROMA DB, AstraDB)


## Facebook AI Similarity Search (FAISS) is a library for efficient similarity search and clustering of dense vectors. It contains algorithms that search in sets of vectors of any size, up to ones that possibly do not fit in RAM. It also contains supporting code for evaluation and parameter tuning.

In [1]:
from langchain_community.document_loaders import TextLoader
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import OllamaEmbeddings
from langchain_text_splitters import CharacterTextSplitter

loader = TextLoader("../speech.txt")
documents = loader.load()
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
docs = text_splitter.split_documents(documents)


In [2]:
docs

[Document(metadata={'source': '../speech.txt'}, page_content='The world must be made safe for democracy. Its peace must be planted upon the tested foundations of political liberty. We have no selfish ends to serve. We desire no conquest, no dominion. We seek no indemnities for ourselves, no material compensation for the sacrifices we shall freely make. We are but one of the champions of the rights of mankind. We shall be satisfied when those rights have been made as secure as the faith and the freedom of nations can make them.\n\nJust because we fight without rancor and without selfish object, seeking nothing for ourselves but what we shall wish to share with all free peoples, we shall, I feel confident, conduct our operations as belligerents without passion and ourselves observe with proud punctilio the principles of right and of fair play we profess to be fighting for.\n\n…'),
 Document(metadata={'source': '../speech.txt'}, page_content='It will be all the easier for us to conduct ou

In [3]:
embeddings = OllamaEmbeddings()
print(embeddings.model)
db = FAISS.from_documents(docs, embeddings)
db

/var/folders/w0/jqmsjv3s0fdf4vt1c98f61x00000gn/T/ipykernel_9437/1707870249.py:1: LangChainDeprecationWarning: The class `OllamaEmbeddings` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaEmbeddings``.
  embeddings = OllamaEmbeddings()


llama2


In [4]:
query = "How does the speaker describe the desired outcome of the war?"
docs = db.similarity_search(query)
docs

[Document(id='5eaaf3ed-12fc-43b5-a591-d0af93e67ecb', metadata={'source': '../speech.txt'}, page_content='It will be all the easier for us to conduct ourselves as belligerents in a high spirit of right and fairness because we act without animus, not in enmity toward a people or with the desire to bring any injury or disadvantage upon them, but only in armed opposition to an irresponsible government which has thrown aside all considerations of humanity and of right and is running amuck. We are, let me say again, the sincere friends of the German people, and shall desire nothing so much as the early reestablishment of intimate relations of mutual advantage between us—however hard it may be for them, for the time being, to believe that this is spoken from our hearts.'),
 Document(id='798a76ba-9724-4774-87a2-389972e9f02e', metadata={'source': '../speech.txt'}, page_content='It is a distressing and oppressive duty, gentlemen of the Congress, which I have performed in thus addressing you. The

In [5]:
docs[0].page_content

'It will be all the easier for us to conduct ourselves as belligerents in a high spirit of right and fairness because we act without animus, not in enmity toward a people or with the desire to bring any injury or disadvantage upon them, but only in armed opposition to an irresponsible government which has thrown aside all considerations of humanity and of right and is running amuck. We are, let me say again, the sincere friends of the German people, and shall desire nothing so much as the early reestablishment of intimate relations of mutual advantage between us—however hard it may be for them, for the time being, to believe that this is spoken from our hearts.'

## As a Retriever

## We can also convert the vectorstore into a Retriever class. This allows us ot easily use it in other LangChain methods, which largely work with retrievers

In [6]:
retriever = db.as_retriever()
retriever.invoke(query)
docs[0].page_content

'It will be all the easier for us to conduct ourselves as belligerents in a high spirit of right and fairness because we act without animus, not in enmity toward a people or with the desire to bring any injury or disadvantage upon them, but only in armed opposition to an irresponsible government which has thrown aside all considerations of humanity and of right and is running amuck. We are, let me say again, the sincere friends of the German people, and shall desire nothing so much as the early reestablishment of intimate relations of mutual advantage between us—however hard it may be for them, for the time being, to believe that this is spoken from our hearts.'

## Similarity search with score

### There are some FAISS specific methods. One of them is similarity_search_with_score, which allows you to return not only the documents but also the distance score of the query to them. The returned distance score is L2 distance. Therefore, a lower score is better

In [7]:
docs_and_score = db.similarity_search_with_score(query)
docs_and_score

[(Document(id='5eaaf3ed-12fc-43b5-a591-d0af93e67ecb', metadata={'source': '../speech.txt'}, page_content='It will be all the easier for us to conduct ourselves as belligerents in a high spirit of right and fairness because we act without animus, not in enmity toward a people or with the desire to bring any injury or disadvantage upon them, but only in armed opposition to an irresponsible government which has thrown aside all considerations of humanity and of right and is running amuck. We are, let me say again, the sincere friends of the German people, and shall desire nothing so much as the early reestablishment of intimate relations of mutual advantage between us—however hard it may be for them, for the time being, to believe that this is spoken from our hearts.'),
  np.float32(15167.131)),
 (Document(id='798a76ba-9724-4774-87a2-389972e9f02e', metadata={'source': '../speech.txt'}, page_content='It is a distressing and oppressive duty, gentlemen of the Congress, which I have performed

In [8]:
embedding_vector = embeddings.embed_query(query)
embedding_vector

[1.8896499872207642,
 -3.095055341720581,
 1.9681845903396606,
 1.4709745645523071,
 -0.8888741731643677,
 0.6569022536277771,
 1.520656943321228,
 -0.4907105565071106,
 0.9173454642295837,
 -2.0103671550750732,
 1.3011105060577393,
 -1.9611570835113525,
 -0.622139573097229,
 1.5270429849624634,
 -0.4982552230358124,
 -2.046572208404541,
 -0.4235247075557709,
 0.06469826400279999,
 0.728234052658081,
 -1.839847445487976,
 -0.6957541108131409,
 -1.03532075881958,
 2.1683218479156494,
 -2.0167531967163086,
 0.7549747228622437,
 -0.9737507104873657,
 0.3398146331310272,
 -1.6769249439239502,
 0.1693202555179596,
 -0.8538368344306946,
 2.0830562114715576,
 -2.852782726287842,
 -2.664320945739746,
 3.7726380825042725,
 2.257633924484253,
 -5.1055121421813965,
 -1.0416245460510254,
 1.3780263662338257,
 -1.2587679624557495,
 -1.3825640678405762,
 -0.46680593490600586,
 -2.042520523071289,
 2.1438777446746826,
 0.3475690186023712,
 0.9232470393180847,
 0.5517717003822327,
 -0.2567218542098999

In [9]:
docs_score = db.similarity_search(embedding_vector)
docs_score

[Document(id='154eeeff-0652-43ba-be33-448515968fa0', metadata={'source': '../speech.txt'}, page_content='To such a task we can dedicate our lives and our fortunes, everything that we are and everything that we have, with the pride of those who know that the day has come when America is privileged to spend her blood and her might for the principles that gave her birth and happiness and the peace which she has treasured. God helping her, she can do no other.'),
 Document(id='5eaaf3ed-12fc-43b5-a591-d0af93e67ecb', metadata={'source': '../speech.txt'}, page_content='It will be all the easier for us to conduct ourselves as belligerents in a high spirit of right and fairness because we act without animus, not in enmity toward a people or with the desire to bring any injury or disadvantage upon them, but only in armed opposition to an irresponsible government which has thrown aside all considerations of humanity and of right and is running amuck. We are, let me say again, the sincere friends 

In [10]:
### Save And Loading
db.save_local("faiss_index")

In [11]:
new_db = FAISS.load_local("faiss_index", embeddings, allow_dangerous_deserialization=True)
docs = new_db.similarity_search(query)

In [12]:
docs

[Document(id='5eaaf3ed-12fc-43b5-a591-d0af93e67ecb', metadata={'source': '../speech.txt'}, page_content='It will be all the easier for us to conduct ourselves as belligerents in a high spirit of right and fairness because we act without animus, not in enmity toward a people or with the desire to bring any injury or disadvantage upon them, but only in armed opposition to an irresponsible government which has thrown aside all considerations of humanity and of right and is running amuck. We are, let me say again, the sincere friends of the German people, and shall desire nothing so much as the early reestablishment of intimate relations of mutual advantage between us—however hard it may be for them, for the time being, to believe that this is spoken from our hearts.'),
 Document(id='798a76ba-9724-4774-87a2-389972e9f02e', metadata={'source': '../speech.txt'}, page_content='It is a distressing and oppressive duty, gentlemen of the Congress, which I have performed in thus addressing you. The